Othman Murad

# Homework 2:  Improve Baseline CNN model and compute metrics for assessing the performance of the CNN-based model

## Notebook Outline

    1. Train Basic Model (From Homework 1)

    2. Saving and Loading Model
    
    3. Metrics Access Performance
    
    4. Hyper-parameter Tuning
    
    5. Overfitting Prevention
    
    6. Compare Performance of Basic and Improved Model

## 1. Train Basic Model (From Homework 1)

In [1]:
# Load the tensorflow, which is a framework for deep learning.
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
# Load numpy library as "np", which can handle large matrices and provides some mathematical functions.
import numpy as np 
# Load pandas as "pd", which is useful when working with data tables. 
import pandas as pd 
# Load random, which provide some randomize functions.
import random
# Load a function pyplot as "plt" to plot figures.
import matplotlib.pyplot as plt
# Load functions to calculate precision, and recall
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


from tensorflow.keras.callbacks import EarlyStopping


# Setup the random seed for reproducibility
seed = 1234
random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)

2024-04-28 19:08:38.290670: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# The root directory of the pmuBAGE data
pmuBAGE_data_dir = "pmuBAGE/data"

# Number of the tensor for voltage and frequency
voltage_tensor_number = 31
frequency_tensor_number = 21

# Load each tensors of voltage events and concatenate them as a big tensor.
voltage_tensor_list = []
for idx in range(voltage_tensor_number):
    voltage_sub_tensor = np.load(f"{pmuBAGE_data_dir}/voltage/voltage_{idx}.npy")
    voltage_tensor_list.append(voltage_sub_tensor)
voltage_tensor = np.concatenate(voltage_tensor_list, axis=0)


# Load each tensors of frequency events and concatenate them as a big tensor.
frequency_tensor_list = []
for idx in range(frequency_tensor_number):
    frequency_sub_tensor = np.load(f"{pmuBAGE_data_dir}/frequency/frequency_{idx}.npy")
    frequency_tensor_list.append(frequency_sub_tensor)
frequency_tensor = np.concatenate(frequency_tensor_list, axis=0)

# Transpose the big tensor as (event_idx, timestamp, PMU_idx, measurements)
voltage_tensor = np.transpose(voltage_tensor, (0, 3, 2, 1))
frequency_tensor = np.transpose(frequency_tensor, (0, 3, 2, 1))

# Print the shape of the voltage event
print(voltage_tensor.shape)
print(frequency_tensor.shape)

(620, 600, 100, 4)
(84, 600, 100, 4)


In [3]:
"""
    Use standardization to pre-process the pmu time series data.
    Input  -> two original tensors: voltage_tensor, frequency_tensor
    Output -> two standardized tensors: voltage_tensor_standardized, frequency_tensor_standardized
    Requirement Details: 
        The tensor shape is (number_of_event, timestamps (600), pmus (100), measurements (4))
        For each time sequence (Single pmu measurement sequence, 600 timestamps), standardize them by Z-Score
        z-score = (x - mean) / std
"""

# Voltage tensor

voltage_mean = np.mean(voltage_tensor, axis=1)
voltage_mean = np.expand_dims(voltage_mean, axis=1)
voltage_std = np.std(voltage_tensor, axis=1)
voltage_std = np.expand_dims(voltage_std, axis=1)
voltage_tensor_standardized = np.nan_to_num((voltage_tensor - voltage_mean) / voltage_std)

# Frequency tensor

frequency_mean = np.mean(frequency_tensor, axis=1)
frequency_mean = np.expand_dims(frequency_mean, axis=1)
frequency_std = np.std(frequency_tensor, axis=1)
frequency_std = np.expand_dims(frequency_std, axis=1)
frequency_tensor_standardized = np.nan_to_num((frequency_tensor - frequency_mean) / frequency_std)

print(voltage_tensor_standardized.shape)
print(frequency_tensor_standardized.shape)


(620, 600, 100, 4)
(84, 600, 100, 4)


In [4]:
# Number of the classes
num_classes = 2

# Number of the voltage and frequency events in the dataset
n_voltage = voltage_tensor_standardized.shape[0]
n_frequency = frequency_tensor_standardized.shape[0]

# Define the labels
# Voltage events' label is defined as: 0
voltage_label = np.array([0] * n_voltage)
# Frequency events' label is defined as: 1
frequency_label = np.array([1] * n_frequency)

"""
    Implement the one-hot encoding on the lablel of of the voltage and frequency event labels.
    Input  -> Original voltage and frequency labels (voltage_label, frequency_label)
    Output -> One-hot encoded voltage and frequency labels (voltage_label_onehot, frequency_label_onthot)
    Voltage label: "0" -> "[1, 0]"
    Frequency label: "1" -> "[0, 1]"
    You can use any library or tool for doing this
"""

voltage_label_onehot = tf.keras.utils.to_categorical(voltage_label, num_classes=num_classes)
frequency_label_onthot = tf.keras.utils.to_categorical(frequency_label, num_classes=num_classes)

# Should be [1, 0]
print(voltage_label_onehot[0])
# Should be [0, 1]
print(frequency_label_onthot[0])
# Should be (620, 2)
print(voltage_label_onehot.shape)
# Should be (84, 2)
print(frequency_label_onthot.shape)

[1. 0.]
[0. 1.]
(620, 2)
(84, 2)


In [5]:
voltage_tensor_standarded_permuted = voltage_tensor_standardized[np.random.permutation(n_voltage)]
frequency_tensor_standarded_permuted = frequency_tensor_standardized[np.random.permutation(n_frequency)]

In [6]:
# Seperate the data to train and test
train_portion = 0.7

# Samples
X_voltage = voltage_tensor_standarded_permuted
X_frequency = frequency_tensor_standarded_permuted
# Labels
y_voltage = voltage_label_onehot
y_frequency = frequency_label_onthot

"""
    Seperate the samples and labels to train and test datasets.
    70% of the voltage and frequency samples and labels are combined as training dataset
    30% remainings are combined as testing dataset
    Input  -> X_voltage, X_frequency, y_voltage, y_frequency
    Output -> X_train, y_train, X_test, y_test
        X_train contains 70% of the X_voltage and X_frequency
        y_train contains 70% of the y_voltage and y_frequency
        X_test contains 30% of the X_voltage and X_frequency
        y_test contains 30% of the y_voltage and y_frequency
"""

# X_train
X_train_voltage = X_voltage[:int(n_voltage * train_portion)] 
X_train_frequency = X_frequency[:int(n_frequency * train_portion)]
X_train = np.concatenate([X_train_voltage, X_train_frequency], axis=0)

# y_train
y_train_voltage = y_voltage[:int(n_voltage * train_portion)] 
y_train_frequency = y_frequency[:int(n_frequency * train_portion)]
y_train = np.concatenate([y_train_voltage, y_train_frequency], axis=0)

# X_test
X_test_voltage = X_voltage[int(n_voltage * train_portion):] 
X_test_frequency = X_frequency[int(n_frequency * train_portion):]
X_test = np.concatenate([X_test_voltage, X_test_frequency], axis=0)

# y_test
y_test_voltage = y_voltage[int(n_voltage * train_portion):] 
y_test_frequency = y_frequency[int(n_frequency * train_portion):]
y_test = np.concatenate([y_test_voltage, y_test_frequency], axis=0)

# Should be (492, 600, 100, 4)
print(X_train.shape)
# Should be (492, 2)
print(y_train.shape)
# Should be (212, 600, 100, 4)
print(X_test.shape)
# Should be (212, 2)
print(y_test.shape)

(492, 600, 100, 4)
(492, 2)
(212, 600, 100, 4)
(212, 2)


In [7]:
def build_model():
    """
        Add more laybers in the model, at least three convolusional layers.
        Then add the Flatten and Dense layers to make the output same with the number of classes.
    """

    model = models.Sequential()
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(600, 100, 4)))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(32, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(32, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Flatten())
    model.add(layers.Dense(32, activation='relu'))
    model.add(layers.Dense(num_classes, activation='softmax'))
    
    return model

model = build_model()

# Define the Loss function
loss_func = tf.keras.losses.CategoricalCrossentropy()

# Define the optimizer and learning rate
lr = 0.01
optimizer = tf.keras.optimizers.Adam(lr)

# Compile the neural network model
model.compile(optimizer=optimizer, loss=loss_func, metrics=['categorical_accuracy'])

# Train the neural network
history = model.fit(X_train, y_train, epochs=10, batch_size=16)

# Evaluate the neural network
loss, accuracy = model.evaluate(X_test, y_test)
print(f"The accuracy of the neural network on the test dataset is: {accuracy}.")

/Users/othmanmurad/Downloads/MLforPMU/MLforPMU/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
31/31 ━━━━━━━━━━━━━━━━━━━━ 24s 693ms/step - categorical_accuracy: 0.8292 - loss: 3.8810
Epoch 2/10
31/31 ━━━━━━━━━━━━━━━━━━━━ 40s 656ms/step - categorical_accuracy: 0.8847 - loss: 0.3790
Epoch 3/10
31/31 ━━━━━━━━━━━━━━━━━━━━ 18s 571ms/step - categorical_accuracy: 0.8847 - loss: 0.3640
Epoch 4/10
31/31 ━━━━━━━━━━━━━━━━━━━━ 19s 613ms/step - categorical_accuracy: 0.8847 - loss: 0.3621
Epoch 5/10
31/31 ━━━━━━━━━━━━━━━━━━━━ 20s 601ms/step - categorical_accuracy: 0.8847 - loss: 0.3622
Epoch 6/10
31/31 ━━━━━━━━━━━━━━━━━━━━ 19s 600ms/step - categorical_accuracy: 0.8847 - loss: 0.3619
Epoch 7/10
31/31 ━━━━━━━━━━━━━━━━━━━━ 19s 605ms/step - categorical_accuracy: 0.8847 - loss: 0.3617
Epoch 8/10
31/31 ━━━━━━━━━━━━━━━━━━━━ 20s 652ms/step - categorical_accuracy: 0.8847 - loss: 0.3614
Epoch 9/10
31/31 ━━━━━━━━━━━━━━━━━━━━ 21s 687ms/step - categorical_accuracy: 0.8847 - loss: 0.3613
Epoch 10/10
31/31 ━━━━━━━━━━━━━━━━━━━━ 19s 605ms/step - categorical_accuracy: 0.8847 - loss: 0.3613
7/7 ━━━━━

## 2. Saving and Loading Model

In [10]:
##-----------------------------------------------------------------------##
##---------------------Students start filling below----------------------##
##-----------------------------------------------------------------------##

"""
    Save trained model to file for future application or further fine-tuning train. 
"""

# Write the code to save the trained model to file.
model.save('homework2.h5')


"""
    Load the model from the file and compile it.
"""

# Write the code to load the model from file and compile it.
model = tf.keras.models.load_model('homework2.h5')



##-----------------------------------------------------------------------##
##------------------------------End filling------------------------------##
##-----------------------------------------------------------------------##

## 3. Metrics Assess Performance

In [12]:
# Get the test samples and labels, and get the model's prediction on test data.
X_test = X_test
y_test = y_test
y_pred = model.predict(X_test)

7/7 ━━━━━━━━━━━━━━━━━━━━ 3s 362ms/step


In [17]:
##-----------------------------------------------------------------------##
##---------------------Students start filling below----------------------##
##-----------------------------------------------------------------------##
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

"""
    Homework 1, only calculate the accuracy of the whole dataset.
    In this task, you required to calculate the accuracy, precision, recall, and F1-score.
"""

# Accuracy
# Precision
# Recall
# F1-Score

y_pred = model.predict(X_test)
accuracy = accuracy_score(np.argmax(y_test, axis=1), np.argmax(y_pred, axis=1))
precision = precision_score(np.argmax(y_test, axis=1), np.argmax(y_pred, axis=1), average='macro')
recall = recall_score(np.argmax(y_test, axis=1), np.argmax(y_pred, axis=1), average='macro')
f1 = f1_score(np.argmax(y_test, axis=1), np.argmax(y_pred, axis=1), average='macro')


print(f"The accuracy is: {accuracy}.")
print(f"The precision is: {precision}.")
print(f"The recall is: {recall}.")
print(f"The f1 score is: {f1}.")

##-----------------------------------------------------------------------##
##------------------------------End filling------------------------------##
##-----------------------------------------------------------------------##

7/7 ━━━━━━━━━━━━━━━━━━━━ 3s 327ms/step
The accuracy is: 0.8773584905660378.
The precision is: 0.4386792452830189.
The recall is: 0.5.
The f1 score is: 0.46733668341708545.


/Users/othmanmurad/Downloads/MLforPMU/MLforPMU/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


## 4. Hyper-parameter Tuning

In [18]:
"""
    Seperate the Training Dataset to Training (80%) and Validation (20%).
    Perform the hyper-parameter tuning to find the best parameter combination.
    
"""

val_portaton = 0.2

n_voltage_train = int(n_voltage * train_portion)
n_frequency_train = int(n_frequency * train_portion)

X_train_voltage = X_voltage[:int(n_voltage * train_portion)] 
X_train_frequency = X_frequency[:int(n_frequency * train_portion)]
X_train = np.concatenate([X_train_voltage, X_train_frequency], axis=0)

# X_val_hp
X_val_hp_voltage = X_train[:int(n_voltage_train * val_portaton)]
X_val_hp_frequency = X_train[n_voltage_train: n_voltage_train + int(n_frequency_train * val_portaton)]
X_val_hp = np.concatenate([X_val_hp_voltage, X_val_hp_frequency], axis=0)

# y_val_hp
y_val_hp_voltage = y_train[:int(n_voltage_train * val_portaton)]
y_val_hp_frequency = y_train[n_voltage_train: n_voltage_train + int(n_frequency_train * val_portaton)]
y_val_hp = np.concatenate([y_val_hp_voltage, y_val_hp_frequency], axis=0)


# X_train_hp
X_train_hp_voltage = X_train[int(n_voltage_train * val_portaton): n_voltage_train]
X_train_hp_frequency = X_train[n_voltage_train + int(n_frequency_train * val_portaton):]
X_train_hp = np.concatenate([X_train_hp_voltage, X_train_hp_frequency], axis=0)

# y_train_hp
y_train_hp_voltage = y_train[int(n_voltage_train * val_portaton): n_voltage_train]
y_train_hp_frequency = y_train[n_voltage_train + int(n_frequency_train * val_portaton):]
y_train_hp = np.concatenate([y_train_hp_voltage, y_train_hp_frequency], axis=0)


# Should be (492, 600, 100, 4)
print(X_train_hp.shape)
# Should be (492, 2)
print(y_train_hp.shape)
# Should be (212, 600, 100, 4)
print(X_val_hp.shape)
# Should be (212, 2)
print(y_val_hp.shape)

(395, 600, 100, 4)
(395, 2)
(97, 600, 100, 4)
(97, 2)


In [21]:
# Setup the random seed for reproducibility
seed = 1234
random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)

In [23]:
# Hyper-parameters
learning_rates = [0.001, 0.01]
batch_sizes = [16, 32]
training_epochs = [10, 20]

##-----------------------------------------------------------------------##
##---------------------Students start filling below----------------------##
##-----------------------------------------------------------------------##


"""
    Try different combination of the hyper-parameters.
    Train on training dataset, test on validation dataset.
    Choose the best hyper-parameter combination to train the final improved model.
"""


best_hyperparameter = {"learning_rate": 0, "batch_size": 0, "training_epoch": 0}
best_accuracy_val = 0.0


for learning_rate in learning_rates:
    for batch_size in batch_sizes:
        for training_epoch in training_epochs:
            print(f"Current hyper-parameters: learning_rate: {learning_rate}, batch_size: {batch_size}, training_epoch: {training_epoch}.")
            
            """
                Train the model under hyper-parameter setting, and evaluate over validation dataset.
            """
            
            """ Filling code below """
            
            # build model
            model_tuning = build_model()

            # Train the model with the above hyper-parameters

            # Define the Loss function
            loss_func = tf.keras.losses.CategoricalCrossentropy()

            # Define the optimizer and learning rate
            
            optimizer = tf.keras.optimizers.Adam(learning_rate)

            # Compile the neural network model
            model_tuning.compile(optimizer=optimizer, loss=loss_func, metrics=['categorical_accuracy'])

            # Train the neural network
            history = model_tuning.fit(X_train_hp, y_train_hp, epochs= training_epoch, batch_size= batch_size)

            
            """ End Filling """
    
            # Evaluate the hyper-parameter tuning neural network
            loss_val, accuracy_val = model_tuning.evaluate(X_val_hp, y_val_hp)
            print(f"Current validation accuracy is: {accuracy_val}.\n")
            
            if accuracy_val > best_accuracy_val:
                best_accuracy_val = accuracy_val
                best_hyperparameter["learning_rate"] = learning_rate
                best_hyperparameter["batch_size"] = batch_size
                best_hyperparameter["training_epoch"] = training_epoch
            tf.keras.backend.clear_session()
            
            
print(f"best validation accuracy is: {best_accuracy_val}")
print(f"best hyper-parameter setting is: {best_hyperparameter}")

##-----------------------------------------------------------------------##
##------------------------------End filling------------------------------##
##-----------------------------------------------------------------------##

Current hyper-parameters: learning_rate: 0.001, batch_size: 16, training_epoch: 10.


/Users/othmanmurad/Downloads/MLforPMU/MLforPMU/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 18s 629ms/step - categorical_accuracy: 0.8315 - loss: 0.7406
Epoch 2/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 16s 621ms/step - categorical_accuracy: 0.8928 - loss: 0.2048
Epoch 3/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 16s 657ms/step - categorical_accuracy: 0.9207 - loss: 0.1808
Epoch 4/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 22s 864ms/step - categorical_accuracy: 0.9394 - loss: 0.1585
Epoch 5/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 20s 772ms/step - categorical_accuracy: 0.9503 - loss: 0.1386
Epoch 6/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 18s 715ms/step - categorical_accuracy: 0.9534 - loss: 0.1344
Epoch 7/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 19s 780ms/step - categorical_accuracy: 0.9550 - loss: 0.1190
Epoch 8/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 15s 567ms/step - categorical_accuracy: 0.9683 - loss: 0.0932
Epoch 9/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 16s 633ms/step - categorical_accuracy: 0.9807 - loss: 0.0915
Epoch 10/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 14s 570ms/step - categorical_accuracy: 0.9748 - loss: 0.0942
4/4 ━━━━━

In [24]:
#####################################
######     Improved Model      ######
#####################################

learning_rate = 0.001
batch_size = 32
training_epoch = 20


model_improved = build_model()

# Train the model with the above hyper-parameters

# Define the Loss function
loss_func = tf.keras.losses.CategoricalCrossentropy()

# Define the optimizer and learning rate

optimizer = tf.keras.optimizers.Adam(learning_rate)

# Compile the neural network model
model_improved.compile(optimizer=optimizer, loss=loss_func, metrics=['categorical_accuracy'])

# Train the neural network
history = model_improved.fit(X_train, y_train, epochs=training_epoch, batch_size= batch_size)

loss, accuracy = model_improved.evaluate(X_test, y_test)

print(f"Current validation accuracy is: {accuracy}.\n")

Epoch 1/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 22s 1s/step - categorical_accuracy: 0.8834 - loss: 0.9956
Epoch 2/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 19s 1s/step - categorical_accuracy: 0.9338 - loss: 0.1858
Epoch 3/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 23s 1s/step - categorical_accuracy: 0.9568 - loss: 0.1298
Epoch 4/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 22s 1s/step - categorical_accuracy: 0.9576 - loss: 0.1001
Epoch 5/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 44s 2s/step - categorical_accuracy: 0.9721 - loss: 0.0732
Epoch 6/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 26s 2s/step - categorical_accuracy: 0.9870 - loss: 0.0533
Epoch 7/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 18s 1s/step - categorical_accuracy: 0.9965 - loss: 0.0325
Epoch 8/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 18s 1s/step - categorical_accuracy: 0.9965 - loss: 0.0187
Epoch 9/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 20s 1s/step - categorical_accuracy: 0.9990 - loss: 0.0137
Epoch 10/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 20s 1s/step - categorical_accuracy: 1.0000 - loss: 0.0100
Epoch 11/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 

## 5. Overfitting Prevention
    Early Stopping to Prevent the overfitting.

In [25]:
# Set the best hyper-parameter from previous tuning. (You may change them based on previous results)
learning_rate = 0.001
batch_size = 32
training_epoch = 20

##-----------------------------------------------------------------------##
##---------------------Students start filling below----------------------##
##-----------------------------------------------------------------------##


"""
    Use the early stopping to prevent the overfitting.
    Useful resource: https://keras.io/api/callbacks/early_stopping/
"""

from tensorflow.keras.callbacks import EarlyStopping


# Build model
model_improved = build_model()
# Define the loss function
loss_func = tf.keras.losses.CategoricalCrossentropy()
# Define the optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate)
# Compile the model
model_improved.compile(optimizer=optimizer, loss=loss_func, metrics=['categorical_accuracy'])

""" Filling code below """

# Define the early stopping callback
early_stopping = EarlyStopping(monitor = 'val_loss',patience = 10, restore_best_weights = True)

""" End Filling """

# Train the model with the tuned hyper-parameters and early-stopping.
history = model_improved.fit(X_train_hp, y_train_hp, validation_data=(X_val_hp, y_val_hp), 
                             epochs=training_epoch, batch_size=batch_size, callbacks=[early_stopping])

##-----------------------------------------------------------------------##
##------------------------------End filling------------------------------##
##-----------------------------------------------------------------------##

Epoch 1/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 25s 2s/step - categorical_accuracy: 0.8423 - loss: 1.8344 - val_categorical_accuracy: 0.8866 - val_loss: 0.2593
Epoch 2/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 15s 1s/step - categorical_accuracy: 0.8947 - loss: 0.2209 - val_categorical_accuracy: 0.9485 - val_loss: 0.1611
Epoch 3/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 15s 1s/step - categorical_accuracy: 0.9400 - loss: 0.1921 - val_categorical_accuracy: 0.9691 - val_loss: 0.1293
Epoch 4/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 14s 1s/step - categorical_accuracy: 0.9584 - loss: 0.1574 - val_categorical_accuracy: 0.9691 - val_loss: 0.1252
Epoch 5/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 19s 2s/step - categorical_accuracy: 0.9656 - loss: 0.1354 - val_categorical_accuracy: 0.9588 - val_loss: 0.1290
Epoch 6/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 18s 1s/step - categorical_accuracy: 0.9681 - loss: 0.1141 - val_categorical_accuracy: 0.9588 - val_loss: 0.1317
Epoch 7/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 19s 1s/step - categorical_accuracy: 0.9732 - loss: 0.0944 - va

## 6. Compare Performance of Basic and Improved Model

In [ ]:
# Precision, recall, F1-score of the basic model

y_pred = model.predict(X_test)
accuracy = accuracy_score(np.argmax(y_test, axis=1), np.argmax(y_pred, axis=1))
precision = precision_score(np.argmax(y_test, axis=1), np.argmax(y_pred, axis=1), average='macro')
recall = recall_score(np.argmax(y_test, axis=1), np.argmax(y_pred, axis=1), average='macro')
f1 = f1_score(np.argmax(y_test, axis=1), np.argmax(y_pred, axis=1), average='macro')

print("Performance of the basic model.")
print(f"The accuracy is: {accuracy}.")
print(f"The precision is: {precision}.")
print(f"The recall is: {recall}.")
print(f"The f1 score is: {f1}.")

In [ ]:
# Precision, recall, F1-score of the improved model

y_pred_improved = model_improved.predict(X_test)
accuracy = accuracy_score(np.argmax(y_test, axis=1), np.argmax(y_pred_improved, axis=1))
precision = precision_score(np.argmax(y_test, axis=1), np.argmax(y_pred_improved, axis=1), average='macro')
recall = recall_score(np.argmax(y_test, axis=1), np.argmax(y_pred_improved, axis=1), average='macro')
f1 = f1_score(np.argmax(y_test, axis=1), np.argmax(y_pred_improved, axis=1), average='macro')

print("Performance of the improved model.")
print(f"The accuracy is: {accuracy}.")
print(f"The precision is: {precision}.")
print(f"The recall is: {recall}.")
print(f"The f1 score is: {f1}.")